<nav class="navbar navbar-default">
  <div class="container-fluid">
    <div class="navbar-header">
      <a class="navbar-brand" href="_Oving7.ipynb">Øving 7</a>
    </div>
    <ul class="nav navbar-nav">
    <li ><a href="Generelt%20om%20dictionary.ipynb">Generelt om dictionary</a></li>
    <li ><a href="Innebygde%20funksjoner%20i%20dictionaries.ipynb">Innebygde funksjoner</a></li>
    <li ><a href="Generelt%20om%20sets.ipynb">Generelt om sets</a></li>
    <li ><a href="Generelt%20om%20filbehandling.ipynb">Generelt om filbehandling</a></li>
    <li ><a href="Osteviruset.ipynb">Osteviruset</a></li>
    <li ><a href="Bursdagsdatabasen.ipynb">Bursdagsdatabasen</a></li>
    <li ><a href="Tallak%20teller%20antall%20tall.ipynb">Tallak teller antall tall</a></li>
    <li ><a href="Opptaksgrenser.ipynb">Opptaksgrenser</a></li>
        <li ><a href="Soke%20i%20tekst.ipynb">Søke i tekst</a></li>
    <li class="active"><a href="Tre%20paa%20rad.ipynb">Tre på rad</a></li>
    </ul>
  </div>
</nav>

# Tre på rad

**Læringsmål:**

* Lister
* Funksjoner 
* Betingelser
* Strenger
 
I denne oppgaven skal du implementere det populære spillet 3 på rad. Spillet er for to spillere; x og o, som plasserer brikker . En spiller vinner om den klarer å få 3 på rad, enten horisontalt, vertikalt eller diagonalt.


**a)** Lag en funksjon som skriver ut brettet, det kan f.eks. se slikt ut om du vil:

```
    1   2   3
  -------------
1 |   |   |   |
  -------------
2 |   |   |   |
  -------------
3 |   |   |   |
  -------------
```

In [ ]:
from math import ceil, floor

FlatMatrix = list[str | None]
Matrix = list[FlatMatrix]


def transform_input(input: any) -> str | None:
    if input is None:
        return None

    return str(input)


# utgangspunkt fra `Matriseaddisjon.ipynb`
def create_matrix(height: int, width: int, data: FlatMatrix) -> Matrix:
    return [[transform_input(data[x + width * y]) for x in range(width)]
            for y in range(height)]


# fra `Matriseaddisjon.ipynb`
def flatten_matrix(data: Matrix) -> FlatMatrix:
    return [x for y in data for x in y]


def pad(string: str, left: int = 1, right: int = 1) -> str:
    return (" " * left) + string + (" " * right)


def center_string(string: str, width: int) -> str:
    difference = width - len(string)
    padding_smaller = difference // 2
    padding_bigger = difference - padding_smaller

    return pad(string, padding_smaller, padding_bigger)


def pad_cells(matrix: Matrix) -> Matrix:
    matrix_height = len(matrix)
    matrix_width = len(matrix[0])
    max_widths = []

    for x in range(len(matrix[0])):
        column_max_width = 0

        for y in range(len(matrix)):
            if matrix[y][x] is None:
                continue

            if len(matrix[y][x]) > column_max_width:
                column_max_width = len(matrix[y][x])

        max_widths.append(column_max_width)

    flat_matrix = flatten_matrix(matrix)
    cells = []

    for i in range(0, len(flat_matrix)):
        x = i % matrix_width

        to_append = " " if flat_matrix[i] is None else flat_matrix[i]
        to_append = pad(to_append.ljust(max_widths[x], " "))

        cells.append(to_append)

    return create_matrix(matrix_height, matrix_width, cells)


def get_prefix(height) -> list[str]:
    upper_bound_plus_one = len(str(height))
    spacer = pad(" ", upper_bound_plus_one - 1, 1)
    prefix = [spacer, spacer]

    for i in range(1, height + 1):
        prefix.append(pad(str(i), upper_bound_plus_one - len(str(i)), 1))
        prefix.append(spacer)

    return prefix


def get_row_separator(first_row_of_cells: FlatMatrix) -> str:
    row_separator = "+"

    for cell in first_row_of_cells:
        row_separator += ("-" * len(cell) + "+")

    return row_separator


def print_header(matrix_width: int, first_row_of_cells: FlatMatrix) -> None:
    for column in range(0, matrix_width):
        column_width = len(first_row_of_cells[column])
        string = str(column + 1)

        print(" " + center_string(string, column_width), end="")

    print("")


def print_matrix(matrix: Matrix) -> None:
    matrix_height = len(matrix)
    matrix_width = len(matrix[0])

    cells = pad_cells(matrix)

    row_separator = get_row_separator(cells[0])
    prefix = get_prefix(matrix_height)

    for i in range(0, matrix_height * 2 + 2):
        print(prefix[i], end="")

        if i == 0:
            print_header(matrix_width, cells[0])
            continue

        if i % 2 != 0:
            print(row_separator)
            continue

        print("|", end="")

        cell_index = ceil((i - 2) / 2)

        for cell in cells[cell_index][:-1]:
            print(f"{cell}|", end="")

        print(f"{cells[cell_index][-1]}|")


In [ ]:
# -- Example 1 --
board_example_1 = create_matrix(1, 3, ["1234567890", None, "1234"])
print_matrix(board_example_1)

print("\n")

# -- Example 2 --
board_example_2 = create_matrix(10, 12, [i**2 for i in range(1, 12 * 10 + 1)])
print_matrix(board_example_2)

print("\n")

# -- Example 3 --
board_example_3 = create_matrix(3, 3, [None for _ in range(0, 9)])
print_matrix(board_example_3)

**b)** Lag en funksjon som sjekker om en spiller har vunnet

In [ ]:
def won_three_in_a_row(matrix: Matrix) -> str | bool | None:
    """
    Returns:
        a) Name of the winner of the game
        b) `False` if the game is not over yet
        c) `None` if the game is over and no one won
    """

    if len(matrix) != 3 or len(matrix[0]) != 3:
        raise ValueError("Matrix is not 3x3")

    has_none = False
    for row in matrix:
        if None in row:
            if not has_none:
                has_none = True
            continue

        if row[0] == row[1] == row[2]:
            return row[0]

    if matrix[0][0] != None and matrix[0][0] == matrix[1][1] == matrix[2][2]:
        return matrix[0][0]

    if matrix[2][0] != None and matrix[2][0] == matrix[1][1] == matrix[0][2]:
        return matrix[2][0]

    if not has_none:
        return False

    return None


In [ ]:
print("navn = vinner, `None` = ikke ferdig, `False` = ingen vinner")

# -- Example 1 --
board_example_1 = create_matrix(3, 3, [
    "X", "X", "X",
    "O", "O", None,
    "O", None, None
])

print_matrix(board_example_1)
print(won_three_in_a_row(board_example_1))
print("\n")

# -- Example 2 --
board_example_2 = create_matrix(3, 3, [
    "X", "X", "O",
    "O", "O", "X",
    "O", None, None
])

print_matrix(board_example_2)
print(won_three_in_a_row(board_example_2))
print("\n")

# -- Example 3 --
board_example_3 = create_matrix(3, 3, [
    "X", "O", "X",
    "O", "O", "X",
    "O", "X", "O"
])

print_matrix(board_example_3)
print(won_three_in_a_row(board_example_3))
print("\n")

# -- Example 4 --
board_example_4 = create_matrix(3, 3, [
    "X", "O", "X",
    "O", "O", "X",
    "O", "X", None
])

print_matrix(board_example_4)
print(won_three_in_a_row(board_example_4))


**c)** Lag en funksjon som tar inn navnene til de to brukerne.

In [67]:
class StopExecution(Exception):

    def _render_traceback_(self):
        return []


def clean_exit() -> None:
    raise StopExecution


def handle_input(raw_input_string: str) -> str:
    if raw_input_string.lower() in ["quit", "exit", "\q"]:
        clean_exit()

    return raw_input_string


def assign_players(number_of_players: int) -> dict[str, str]:
    players: dict[str, str] = {}

    while len(players) < NUMBER_OF_PLAYERS:
        n = len(players) + 1
        
        player_name = handle_input(input(f"Spiller {n}: "))

        if player_name in players.keys():
            print(f"Navnet '{player_name}' er allerede tatt. Prøv på nytt.")
            continue

        player_alias = handle_input(input(f"Alias for {player_name} (spiller {n}): "))

        if player_alias in players.values():
            print(f"Aliaset '{player_alias}' er allerede tatt. Prøv på nytt.")
            continue

        players.update({player_name: player_alias})

    return players


In [ ]:
NUMBER_OF_PLAYERS = 2
players = assign_players(NUMBER_OF_PLAYERS)

for name, alias in players.items():
    print(f"Spiller {name} har alias {alias}")


**d)** Lag en funksjon som sjekker om et trekk er lovlig, altså at det ikke finnes andre brikker der.

In [75]:
def is_legal_move(board: Matrix, to_coordinate: (int, int)) -> bool:
	x, y = to_coordinate
	
	if board[y - 1][x - 1] is not None:
		return False

	return True

In [ ]:
print_matrix(board_example_1)
print(f"Kan plassere brikke i (2, 3): {is_legal_move(board_example_1, (2, 3))}")
print("\n")

print_matrix(board_example_3)
print(f"Kan plassere brikke i (2, 3): {is_legal_move(board_example_3, (2, 3))}")


**e)** Lag en funksjon som sjekker at input fra brukeren er riktig, altså at man ikke skriver inn rare tegn, eller skriver inn koordinater utenfor spillebrettet.

In [69]:
def validate_coordinate_input(matrix: Matrix, coordinate: (int, int)) -> bool:
    if len(coordinate) != 2:
        return False

    if not isinstance(coordinate[0], int) or not isinstance(coordinate[1], int):
        return False

    matrix_height = len(matrix)
    matrix_width = len(matrix[0])

    if 1 <= coordinate[0] <= matrix_width and 1 <= coordinate[1] <= matrix_height:
        return True

    return False


In [ ]:
print_matrix(board_example_1)
print(f"Er (3, 3)    godkjent? {validate_coordinate_input(board_example_1, (3, 3))}")
print(f"Er (1, 1)    godkjent? {validate_coordinate_input(board_example_1, (1, 1))}")
print(f"Er (0, 1)    godkjent? {validate_coordinate_input(board_example_1, (0, 1))}")
print(f"Er (2, 3, 3) godkjent? {validate_coordinate_input(board_example_1, (2, 3, 3))}")
print(f"Er (2, 2.1)  godkjent? {validate_coordinate_input(board_example_1, (2, 2.1))}")
print(f"Er (4, 2)    godkjent? {validate_coordinate_input(board_example_1, (4, 2))}")
print(f"Er (2, -2)   godkjent? {validate_coordinate_input(board_example_1, (2, -2))}")
print(f"Er (2, 'a')  godkjent? {validate_coordinate_input(board_example_1, ('a', 2))}")


**f)** Sett dette sammen til et fullverdig 3 på rad spill!

In [84]:
## +-------------------------------+
## |                               |
## | SE EGEN FIL: `Tre paa rad.py` |
## |                               |
## +-------------------------------+

from math import ceil, floor
from re import sub

FlatMatrix = list[str | None]
Matrix = list[FlatMatrix]

NUMBER_OF_PLAYERS = 3
WINNER_LENGTH = 2
BOARD_HEIGHT = 3
BOARD_WIDTH = 5


class StopExecution(Exception):
    def _render_traceback_(self):
        return []


def clean_exit() -> None:
    raise StopExecution


def handle_input(raw_input_string: str) -> str:
    if raw_input_string.lower() in ["quit", "exit", "\q"]:
        clean_exit()

    return raw_input_string


def to_str_or_none(input: any) -> str | None:
    if input is None:
        return None

    return str(input)


# utgangspunkt fra `Matriseaddisjon.ipynb`
def create_matrix(height: int, width: int, data: FlatMatrix) -> Matrix:
    return [
        [to_str_or_none(data[x + width * y]) for x in range(width)]
        for y in range(height)
    ]


# fra `Matriseaddisjon.ipynb`
def flatten_matrix(data: Matrix) -> FlatMatrix:
    return [x for y in data for x in y]


def pad(string: str, left: int = 1, right: int = 1) -> str:
    return (" " * left) + string + (" " * right)


def center_string(string: str, width: int) -> str:
    difference = width - len(string)
    padding_smaller = difference // 2
    padding_bigger = difference - padding_smaller

    return pad(string, padding_smaller, padding_bigger)


def pad_cells(matrix: Matrix) -> Matrix:
    matrix_height = len(matrix)
    matrix_width = len(matrix[0])
    max_widths = []

    for x in range(len(matrix[0])):
        column_max_width = 0

        for y in range(len(matrix)):
            if matrix[y][x] is None:
                continue

            if len(matrix[y][x]) > column_max_width:
                column_max_width = len(matrix[y][x])

        max_widths.append(column_max_width)

    flat_matrix = flatten_matrix(matrix)
    cells = []

    for i in range(0, len(flat_matrix)):
        x = i % matrix_width

        to_append = " " if flat_matrix[i] is None else flat_matrix[i]
        to_append = pad(to_append.ljust(max_widths[x], " "))

        cells.append(to_append)

    return create_matrix(matrix_height, matrix_width, cells)


def get_prefix(height) -> list[str]:
    upper_bound_plus_one = len(str(height))
    spacer = pad(" ", upper_bound_plus_one - 1, 1)

    prefix = [spacer, spacer]

    for i in range(1, height + 1):
        prefix.append(pad(str(i), upper_bound_plus_one - len(str(i)), 1))
        prefix.append(spacer)

    return prefix


def get_row_separator(first_row_of_cells: FlatMatrix) -> str:
    row_separator = "+"

    for cell in first_row_of_cells:
        row_separator += "-" * len(cell) + "+"

    return row_separator


def print_header(matrix_width: int, first_row_of_cells: FlatMatrix) -> None:
    for column in range(0, matrix_width):
        column_width = len(first_row_of_cells[column])
        string = str(column + 1)

        print(" " + center_string(string, column_width), end="")

    print("")


def print_matrix(matrix: Matrix) -> None:
    matrix_height = len(matrix)
    matrix_width = len(matrix[0])

    prefix = get_prefix(matrix_height)
    cells = pad_cells(matrix)

    row_separator = get_row_separator(cells[0])

    for i in range(0, matrix_height * 2 + 2):
        print(prefix[i], end="")

        if i == 0:
            print_header(matrix_width, cells[0])
            continue

        if i % 2 != 0:
            print(row_separator)
            continue

        print("|", end="")

        cell_index = ceil((i - 2) / 2)

        for cell in cells[cell_index][:-1]:
            print(f"{cell}|", end="")

        print(f"{cells[cell_index][-1]}|")


def all_equal(list: list) -> bool:
    return len(set(list)) == 1 and list[0] is not None


def get_winner(matrix: Matrix) -> str | bool | None:
    """
    str   => aliaset til vinneren
    False => spillet er ikke ferdig
    None  => spillet er uavgjort
    """

    matrix_height = len(matrix)
    matrix_width = len(matrix[0])

    if matrix_height != BOARD_HEIGHT or matrix_width != BOARD_WIDTH:
        raise ValueError(f"Matrix is not {BOARD_WIDTH}x{BOARD_HEIGHT}")

    has_none = False
    shift = WINNER_LENGTH - 1

    # _ _ _
    # X X X
    # _ _ _
    for y in range(0, matrix_height):
        if None in matrix[y]:
            has_none = True

        for x in range(0, matrix_width - shift):
            if matrix[y][x] == None:
                continue

            if all_equal([matrix[y][x + i] for i in range(0, WINNER_LENGTH)]):
                return matrix[y][x]

    if not has_none:
        return False

    # _ X _
    # _ X _
    # _ X _
    for x in range(0, matrix_width):
        for y in range(0, matrix_height - shift):
            if matrix[y][x] == None:
                continue

            if all_equal([matrix[y + i][x] for i in range(0, WINNER_LENGTH)]):
                return matrix[y][x]

    # X _ _
    # _ X _
    # _ _ X
    for y in range(0, matrix_height - shift):
        for x in range(0, matrix_width - shift):
            if matrix[y][x] == None:
                continue

            if all_equal([matrix[y + i][x + i] for i in range(0, WINNER_LENGTH)]):
                return matrix[y][x]

    # _ _ X
    # _ X _
    # X _ _
    for y in range(0, matrix_height - shift):
        for x in range(shift, matrix_width):
            if matrix[y][x] == None:
                continue

            if all_equal([matrix[y + i][x - i] for i in range(0, WINNER_LENGTH)]):
                return matrix[y][x]

    return None


def assign_players(number_of_players: int) -> dict[str, str]:
    players: dict[str, str] = {}

    while len(players) < NUMBER_OF_PLAYERS:
        n = len(players) + 1

        name = handle_input(input(f"Spiller {n}: "))

        if name in players.keys():
            print(f"Navnet '{name}' er allerede tatt. Prøv på nytt.")
            continue

        alias = handle_input(input(f"Alias for {name} (spiller {n}): "))

        if alias in players.values():
            print(f"Aliaset '{alias}' er allerede tatt. Prøv på nytt.")
            continue

        players.update({name: alias})

    return players


def is_legal_move(board: Matrix, to_coordinate: (int, int)) -> bool:
    x, y = to_coordinate

    if board[y - 1][x - 1] is not None:
        return False

    return True


def validate_coordinate_input(matrix: Matrix, coordinate: (int, int)) -> bool:
    if len(coordinate) != 2:
        return False

    x, y = coordinate

    if not isinstance(x, int) or not isinstance(y, int):
        return False

    matrix_height = len(matrix)
    matrix_width = len(matrix[0])

    if 1 <= x <= matrix_width and 1 <= y <= matrix_height:
        return True

    return False


def get_next_coordinate(board: Matrix) -> (int, int):
    coordinate = None

    while coordinate == None:
        raw_coordinate = handle_input(input("Velg koordinat (x, y): "))
        cleaned_coordinate = sub(r"[^0-9,]", "", raw_coordinate)
        x_y_split = cleaned_coordinate.split(",")

        if len(x_y_split) != 2:
            print(f"Ugyldig koordinat {raw_coordinate}. Prøv igjen.")
            continue

        try:
            x = int(x_y_split[0])
            y = int(x_y_split[1])

            if not validate_coordinate_input(board, (x, y)):
                print(f"Ugyldig koordinat {raw_coordinate}. Prøv igjen.")
                continue

            if not is_legal_move(board, (x, y)):
                print(f"Ugyldig trekk {raw_coordinate}. Prøv igjen.")
                continue

            coordinate = (x, y)
        except:
            print(f"Ugyldig koordinat {raw_coordinate}. Prøv igjen.")
            continue

    return coordinate


empty_data = [None for _ in range(0, BOARD_HEIGHT * BOARD_WIDTH)]
board = create_matrix(BOARD_HEIGHT, BOARD_WIDTH, empty_data)

winner = get_winner(board)
players = assign_players(2)

i = 0
while winner == None:
    i += 1

    for name, alias in players.items():
        print("\n\n")
        print_matrix(board)
        print(f"Runde {i} - {WINNER_LENGTH} på rad - {len(players)} spillere\n")
        print(f"Spiller {name} ({alias}) sin tur")

        x, y = get_next_coordinate(board)

        board[y - 1][x - 1] = alias

        winner = get_winner(board)

        if winner != None:
            break

if winner == False:
    print("\n\n")
    print_matrix(board)
    print("\nSpillet er ferdig. Det ble uavgjort.")

winner_name = list(players.keys())[list(players.values()).index(winner)]

print("\n\n")
print_matrix(board)
print(f"\nSpillet er ferdig. Spiller {winner_name} ({winner}) vant!")




    1   2   3 
  +---+---+---+
1 |   |   |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+
3 |   |   |   |
  +---+---+---+

Runde 1


Spiller a (a) sin tur


    1   2   3 
  +---+---+---+
1 |   |   |   |
  +---+---+---+
2 |   | a |   |
  +---+---+---+
3 |   |   |   |
  +---+---+---+

Runde 1


Spiller b (b) sin tur
Ugyldig trekk 2,2. Prøv igjen.


    1   2   3 
  +---+---+---+
1 | b |   |   |
  +---+---+---+
2 |   | a |   |
  +---+---+---+
3 |   |   |   |
  +---+---+---+

Runde 2


Spiller a (a) sin tur
Ugyldig trekk 1,1. Prøv igjen.
Ugyldig trekk 1,1. Prøv igjen.


    1   2   3 
  +---+---+---+
1 | b |   |   |
  +---+---+---+
2 | a | a |   |
  +---+---+---+
3 |   |   |   |
  +---+---+---+

Runde 2


Spiller b (b) sin tur


    1   2   3 
  +---+---+---+
1 | b | b |   |
  +---+---+---+
2 | a | a |   |
  +---+---+---+
3 |   |   |   |
  +---+---+---+

Runde 3


Spiller a (a) sin tur
